# Imports and Setup

In [ ]:
# %load_ext cudf.pandas
%matplotlib inline

import os
import ast
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from scipy import stats
from pprint import PrettyPrinter
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from visualize._utils import experiment_parameter

sns.set_style("whitegrid")
pp = PrettyPrinter(indent=4, width=100, compact=True)

# Color and Marker Legend
Returns the common legend code used for all plots.

In [ ]:
# 1. Marker and Dataset color codes
def legend_code():
    markers = {
        "<": "Naive",
        ">": "Linear",
        "p": "Feedforward",
        "s": "Transformer",
        "P": "Attention",
        "o": "LSTM",
        "*": "CTRNN",
        "X": "Liquid",
    }

    model_labels = {
        "NaivePredictor": "Naive",
        "LinearRegression": "Linear",
        "FeatureFFNN": "Feedforward",
        "NeuralTransformer": "Transformer",
        "PureAttention": "Attention",
        "NetworkLSTM": "LSTM",
        "NetworkCTRNN": "CTRNN",
        "LiquidCfC": "Liquid",
    }

    dataset_labels = {
        "Kato2015": "Kato (2015)",
        "Nichols2017": "Nichols (2017)",
        "Skora2018": "Skora (2018)",
        "Kaplan2020": "Kaplan (2020)",
        "Yemini2021": "Yemini (2021)",
        "Uzel2022": "Uzel (2022)",
        "Flavell2023": "Flavell (2023)",
        "Leifer2023": "Leifer (2023)",
    }

    marker_colors = sns.color_palette("tab10", n_colors=len(markers))

    # Create custom markers for models
    marker_legend = [
        Line2D([0], [0], marker=m, color=marker_colors[i], label=l, linestyle="None")
        for i, (m, l) in enumerate(markers.items())
    ]

    # Plot the marker legends
    fig, axs = plt.subplots(1, 2, figsize=(4, 1))

    # Plot marker legend on the left subplot
    axs[0].legend(handles=marker_legend, loc="center", title="Model")
    # Legend title italic
    axs[0].get_legend().get_title().set_fontsize("large")
    axs[0].get_legend().get_title().set_fontstyle("italic")
    axs[0].axis("off")

    color_palette = sns.color_palette("tab10", n_colors=len(dataset_labels))
    # Add black color to the end of color palette
    color_palette.append((0, 0, 0))

    # Create rectangular color patches for datasets
    color_legend = [
        Patch(facecolor=c, edgecolor=c, label=l) for c, l in zip(color_palette, dataset_labels)
    ]

    # Plot color legend on the right subplot
    axs[1].legend(handles=color_legend, loc="center", title="Experimental datasets")
    axs[1].get_legend().get_title().set_fontsize("large")
    axs[1].get_legend().get_title().set_fontstyle("italic")
    axs[1].axis("off")

    dataset_color_code = {
        dataset: color for dataset, color in zip(dataset_labels.keys(), color_palette)
    }
    model_marker_code = {model: marker for model, marker in zip(markers.values(), markers.keys())}
    model_color_code = {model: color for model, color in zip(markers.values(), marker_colors)}

    plt.show()

    leg_code = {
        "dataset_color_code": dataset_color_code,
        "model_marker_code": model_marker_code,
        "model_color_code": model_color_code,
        "color_legend": color_legend,
        "dataset_labels": dataset_labels,
        "marker_colors": marker_colors,
        "marker_legend": marker_legend,
        "model_labels": model_labels,
    }

    return leg_code


# Usage example
leg_code = legend_code()
pp.pprint(leg_code)

# Dataset information

In [ ]:
# %%cudf.pandas.profile


# 2. Plotting summary information about the datasets
def dataset_information(path_dict, legend_code):
    """
    path_dict: dictionary with the path to train_dataset_info,
        validation_dataset_info and combined_dataset_info.
    """

    # ### LOAD IN DATASET INFORMATION ###
    train_dataset_info = pd.read_csv(
        path_dict["train_dataset_info"],
        converters={"neurons": ast.literal_eval},
    )
    val_dataset_info = pd.read_csv(
        path_dict["val_dataset_info"],
        converters={"neurons": ast.literal_eval},
    )
    combined_dataset_info = pd.read_csv(
        path_dict["combined_dataset_info"],
        converters={"neurons": ast.literal_eval},
    )

    train_dataset_info["total_time_steps"] = (
        train_dataset_info["train_time_steps"] + val_dataset_info["val_time_steps"]
    )
    train_dataset_info["time_steps_per_neuron"] = (
        train_dataset_info["total_time_steps"] / train_dataset_info["num_neurons"]
    )
    amount_of_data_distribution = (
        train_dataset_info[["dataset", "total_time_steps"]]
        .groupby("dataset")
        .sum()
        .sort_values(by="total_time_steps", ascending=False)
    )
    amount_of_data_distribution["percentage"] = (
        amount_of_data_distribution["total_time_steps"]
        / amount_of_data_distribution["total_time_steps"].sum()
    )

    # ########### SET UP FOR FIGURES ###########
    # Get color code and legend from legend_code
    dataset_color_code = legend_code["dataset_color_code"]
    dataset_labels = legend_code["dataset_labels"]
    color_legend = legend_code["color_legend"]

    # Initialize figure
    fig = plt.figure(figsize=(20, 7))
    gs = gridspec.GridSpec(2, 4, height_ratios=[1, 1], width_ratios=[1, 1, 1, 0.25])

    # Assigning the subplots to positions in the grid
    ax1 = plt.subplot(gs[0, 0])  # Top left, 'Number of worms analyzed' pie chart
    ax2 = plt.subplot(gs[0, 1])  # Top right, 'Number of neurons per worm' bar plot
    ax3 = plt.subplot(
        gs[1, 0]
    )  # Bottom left, 'Total duration of recorded neural activity' pie chart
    ax4 = plt.subplot(
        gs[1, 1]
    )  # Bottom middle, 'Duration of recorded neural activity per worm' bar plot
    ax5 = plt.subplot(gs[0, 3])  # Bottom right, legend
    ax6 = plt.subplot(gs[0, 2])
    ax7 = plt.subplot(gs[1, 2:4])

    # ########### FOR WORMS PIE CHART ###############
    num_worms_per_dataset = combined_dataset_info[["dataset", "original_index"]]
    # Count the unique 'original_index' for each 'dataset'
    num_worms_per_dataset = (
        num_worms_per_dataset.groupby("dataset")["original_index"]
        .nunique()
        .reset_index(name="num_worms")
    )
    # Calculate the percentage for each dataset
    num_worms_per_dataset["percentage"] = (
        num_worms_per_dataset["num_worms"] / num_worms_per_dataset["num_worms"].sum()
    )
    # Sort the values by percentage in descending order
    num_worms_per_dataset = num_worms_per_dataset.sort_values(by="percentage", ascending=False)
    worm_count_label = num_worms_per_dataset["num_worms"][:7].tolist() + [""]
    # Plotting the worms per dataset pie chart
    ax1.pie(
        num_worms_per_dataset["num_worms"],
        labels=[f"{percentage:.1%}" for percentage in num_worms_per_dataset["percentage"]],
        labeldistance=1.075,
        startangle=45,
        colors=[dataset_color_code[dataset] for dataset in num_worms_per_dataset["dataset"]],
    )
    ax1.pie(
        num_worms_per_dataset["num_worms"],
        labels=[f"{n}" for n in worm_count_label],
        labeldistance=0.70,
        startangle=45,
        colors=[dataset_color_code[dataset] for dataset in num_worms_per_dataset["dataset"]],
    )
    ax1.set_title("(A) Number of worms in dataset", fontsize=14)

    # ########### NEURON POPULATION DISTRIBUTION BAR PLOT ###############
    ax2 = plt.subplot(gs[0, 1])  # Subplot for 'Number of neurons per worm' bar plot
    # Compute data for the neuron population distribution bar plot
    neuron_pop_stats = (
        train_dataset_info.groupby("dataset")["num_neurons"]
        .agg(["mean", "sem"])
        .reset_index()
        .sort_values(by="mean", ascending=False)
    )
    neuron_pop_colors = neuron_pop_stats["dataset"].apply(
        lambda x: dataset_color_code.get(x, "grey")
    )
    ax2.bar(
        neuron_pop_stats["dataset"],
        neuron_pop_stats["mean"],
        yerr=2 * neuron_pop_stats["sem"],
        color=neuron_pop_colors,
        capsize=5,
    )
    # Add a dashed horizontal line at y=302
    ax2.axhline(y=302, color="black", linestyle="dashed", linewidth=1, alpha=0.5)
    # Annotate the line
    # Use the right edge of the subplot for text annotation to prevent overflow
    right_edge = ax2.get_xlim()[1]
    ax2.text(
        right_edge,  # x position at the right edge
        302,  # y position at the line
        "Number of neurons in C. elegans hermaphrodite",
        horizontalalignment="right",  # Align text to the right
        fontsize=10,
        fontstyle="italic",
    )
    ax2.set_title("(B) Number of recorded neurons per worm", fontsize=14)
    ax2.set_ylabel("Neuron population size")
    ax2.set_xticklabels(neuron_pop_stats["dataset"], rotation=45, ha="right")
    ax2.set_xticks([])  # Delete xticks

    # ########### TOTAL DURATION OF RECORDED NEURAL ACTIVITY PIE CHART ###############
    ax3 = plt.subplot(
        gs[1, 0]
    )  # Subplot for 'Total duration of recorded neural activity' pie chart
    # Compute data for total duration pie chart
    total_duration_stats = (
        train_dataset_info.groupby("dataset")["total_time_steps"]
        .sum()
        .reset_index()
        .sort_values(by="total_time_steps", ascending=False)
    )
    total_duration_stats["percentage"] = (
        total_duration_stats["total_time_steps"] / total_duration_stats["total_time_steps"].sum()
    )

    # Determine the smallest slice
    smallest_slice_index = total_duration_stats["percentage"].idxmin()

    # Generate labels, omitting the smallest slice
    labels = [
        f"{percentage:.1%}" if i != smallest_slice_index else ""
        for i, percentage in enumerate(total_duration_stats["percentage"])
    ]

    total_duration_colors = total_duration_stats["dataset"].apply(
        lambda x: dataset_color_code.get(x, "grey")
    )
    # Plotting the total duration pie chart
    ax3.pie(
        total_duration_stats["total_time_steps"],
        labels=labels,
        labeldistance=1.075,
        startangle=90,
        colors=total_duration_colors,
    )
    ax3.set_title("(C) Total duration of recorded neural activity", fontsize=14)

    # ########### DURATION OF RECORDED NEURAL ACTIVITY PER WORM BAR PLOT ###############
    ax4 = plt.subplot(
        gs[1, 1]
    )  # Subplot for 'Duration of recorded neural activity per worm' bar plot
    # Compute data for recording duration bar plot
    recording_duration_stats = (
        train_dataset_info.groupby("dataset")["total_time_steps"]
        .agg(["mean", "sem"])
        .reset_index()
        .sort_values(by="mean", ascending=False)
    )
    recording_duration_colors = recording_duration_stats["dataset"].apply(
        lambda x: dataset_color_code.get(x, "grey")
    )
    ax4.bar(
        recording_duration_stats["dataset"],
        recording_duration_stats["mean"],
        yerr=2 * recording_duration_stats["sem"],
        color=recording_duration_colors,
        capsize=5,
    )
    # Add a dashed horizontal line at y=3600
    ax4.axhline(y=3600, color="black", linestyle="dashed", linewidth=1, alpha=0.5)
    # Annotate the line
    # Use the right edge of the subplot for text annotation to prevent overflow
    right_edge = ax4.get_xlim()[1]
    ax4.text(
        right_edge,  # x position at the right edge
        3600,  # y position at the line
        "3600 seconds = 1 hour of calcium imaging",
        horizontalalignment="right",  # Align text to the right
        fontsize=10,
        fontstyle="italic",
    )
    ax4.set_title("(D) Duration of recorded neural activity per worm", fontsize=14)
    ax4.set_ylabel("Time (s)")
    ax4.set_xticklabels(recording_duration_stats["dataset"], rotation=45, ha="right")
    ax4.set_xticks([])  # Delete xticks

    # ########### TIME STEPS PER NEURON BAR PLOT ###############
    ax6 = plt.subplot(gs[0, 2])  # Subplot for 'Number of time steps per recorded neuron' bar plot
    # Compute data for time steps per neuron bar plot
    tsn_stats = (
        train_dataset_info.groupby("dataset")["time_steps_per_neuron"]
        .agg(["mean", "sem"])
        .reset_index()
        .sort_values(by="mean", ascending=False)
    )
    tsn_colors = tsn_stats["dataset"].apply(lambda x: dataset_color_code.get(x, "grey"))
    ax6.bar(
        tsn_stats["dataset"],
        tsn_stats["mean"],
        yerr=2 * tsn_stats["sem"],
        color=tsn_colors,
        capsize=5,
    )
    # Add a dashed horizontal line at y=100
    ax6.axhline(y=100, color="black", linestyle="dashed", linewidth=1, alpha=0.5)
    # Annotate the line
    # Use the right edge of the subplot for text annotation to prevent overflow
    right_edge = ax6.get_xlim()[1]
    ax6.text(
        right_edge,  # x position at the right edge
        100,  # y position at the line
        "Sequence length $L=100$ was used in our experiments",
        horizontalalignment="right",  # Align text to the right
        fontsize=10,
        fontstyle="italic",
    )
    ax6.set_title("(E) Number of time steps per recorded neuron", fontsize=14)
    ax6.set_ylabel("Time steps per neuron")
    ax6.set_xticklabels(tsn_stats["dataset"], rotation=45, ha="right")
    ax6.set_xticks([])  # Delete xticks

    # ########### SAMPLING INTERVAL BAR PLOT ###############
    ax7 = plt.subplot(
        gs[1, 2:4]
    )  # Subplot for 'Sampling interval of recorded neural activity' bar plot
    # Compute data for the sampling interval bar plot
    dt_stats = (
        train_dataset_info.groupby("dataset")["original_median_dt"]
        .agg(["mean", "sem"])
        .reset_index()
        .sort_values(by="mean", ascending=False)
    )
    dt_colors = dt_stats["dataset"].apply(lambda x: dataset_color_code.get(x, "grey"))
    ax7.bar(
        dt_stats["dataset"],
        dt_stats["mean"],
        yerr=2 * dt_stats["sem"],
        color=dt_colors,
        capsize=5,
    )
    # Add a dashed horizontal line at y=1.0
    ax7.axhline(y=1.0, color="black", linestyle="dashed", linewidth=1, alpha=0.5)
    # Annotate the line
    # Use the right edge of the subplot for text annotation to prevent overflow
    right_edge = ax7.get_xlim()[1]
    ax7.text(
        right_edge,  # x position at the right edge
        1.0,  # y position at the line
        "We downsampled all data to $\Delta s = 1.0s$ (1 Hz)",
        horizontalalignment="right",  # Align text to the right
        fontsize=10,
        fontstyle="italic",
    )
    ax7.set_title("(F) Sampling interval of recorded neural activity", fontsize=14)
    ax7.set_ylabel(r"Mean sampling interval ($\Delta$s)")
    ax7.set_xticklabels(dt_stats["dataset"], rotation=45, ha="right")
    ax7.set_xticks([])  # Delete xticks

    # ########### LEGEND SUBPLOT ###############
    ax5 = plt.subplot(gs[0, 3])  # Subplot for legend
    ax5.legend(
        handles=color_legend,
        labels=dataset_labels.values(),  # DEBUG
        loc="center",
        title="Experimental datasets",
        fontsize=11,
        title_fontsize=12,
    )
    ax5.get_legend().get_title().set_fontstyle("italic")
    ax5.axis("off")

    plt.tight_layout()
    plt.show()

    # Construct and return the dataset_info dictionary with all the computed stats
    dataset_info = {
        "train_dataset_info": train_dataset_info,
        "amount_of_data_distribution": amount_of_data_distribution,
        "num_worms_per_dataset": num_worms_per_dataset,
        "total_duration_stats": total_duration_stats,
        "neuron_pop_stats": neuron_pop_stats,
        "recording_duration_stats": recording_duration_stats,
        "tsn_stats": tsn_stats,
        "dt_stats": dt_stats,
    }

    return dataset_info

In [ ]:
# Usage example
fig1_path_dict = {
    "train_dataset_info": "/om2/user/qsimeon/worm-graph/data/train_AllExperimental/train_dataset_info.csv",  # Path to train dataset info => extract number of train time steps
    "val_dataset_info": "/om2/user/qsimeon/worm-graph/data/validation_AllExperimental/val_dataset_info.csv",  # Path to val. dataset info => extract number of val. time steps
    "combined_dataset_info": "/om2/user/qsimeon/worm-graph/data/combined_AllExperimental/combined_dataset_info.csv",  # Path to combined dataset info => extract total number of worms and time step interval
}

# NOTE: Error bars on bar plots show +/- 2 SEM
dataset_info = dataset_information(path_dict=fig1_path_dict, legend_code=leg_code)

# Scaling Law Figures

In [ ]:
# %%cudf.pandas.profile


def get_results_df(results_root_dir):
    """
    Recursively traverses the experiment directories, reads train_metrics.csv and
    validation_loss_per_dataset.csv files, and combines the data into a comprehensive DataFrame.

    Parameters:
    - results_root_dir: The root directory where the experiment directories are located.

    Returns:
    - A DataFrame containing all the combined results from the experiments.
    """

    def recursive_search(path, results):
        """
        Helper function to perform a recursive search for csv files in the directory tree.
        """
        for entry in os.scandir(path):
            if entry.is_dir() and any(os.scandir(entry.path)):
                recursive_search(entry.path, results)
            elif entry.is_file():
                if entry.name == "train_metrics.csv":
                    train_metrics_path = entry.path
                    validation_loss_path = os.path.join(
                        os.path.dirname(os.path.dirname(train_metrics_path)),
                        "analysis",
                        "validation_loss_per_dataset.csv",
                    )

                    if os.path.exists(validation_loss_path):
                        # Read the csv files
                        train_metrics_df = pd.read_csv(train_metrics_path)
                        validation_loss_df = pd.read_csv(validation_loss_path)

                        # Extract parameters using the provided experiment_parameter function
                        exp_path = os.path.dirname(os.path.dirname(train_metrics_path))
                        experiment_seed = experiment_parameter(exp_path, "experiment_seed")[0]
                        train_dataset = experiment_parameter(exp_path, "train_dataset")[0]
                        num_worms = experiment_parameter(exp_path, "num_worms")[0]
                        num_time_steps = experiment_parameter(exp_path, "num_time_steps")[0]
                        num_named_neurons = experiment_parameter(exp_path, "num_named_neurons")[0]
                        time_steps_per_neuron = experiment_parameter(
                            exp_path, "time_steps_per_neuron"
                        )[0]
                        train_split_first = experiment_parameter(exp_path, "train_split_first")[0]
                        hidden_size = experiment_parameter(exp_path, "hidden_size")[0]
                        batch_size = experiment_parameter(exp_path, "batch_size")[0]
                        seq_len = experiment_parameter(exp_path, "seq_len")[0]
                        resample_dt = experiment_parameter(exp_path, "resample_dt")[0]
                        time_last_epoch = experiment_parameter(exp_path, "time_last_epoch")[0]
                        computation_flops = experiment_parameter(exp_path, "computation_flops")[0]
                        num_parameters = experiment_parameter(exp_path, "num_parameters")[0]
                        model_type = experiment_parameter(exp_path, "model")[0]

                        # Calculate the min_val_loss and val_baseline
                        min_val_loss = train_metrics_df["val_loss"].min()
                        val_baseline = train_metrics_df["val_baseline"].mean()

                        # Append data to the results list
                        for _, row in validation_loss_df.iterrows():
                            results.append(
                                {
                                    "experiment_ID": os.path.basename(exp_path),
                                    "experiment_seed": experiment_seed,
                                    "train_dataset": train_dataset,
                                    "num_worms": num_worms,
                                    "num_time_steps": num_time_steps,
                                    "num_named_neurons": num_named_neurons,
                                    "time_steps_per_neuron": time_steps_per_neuron,
                                    "train_split_first": train_split_first,
                                    "hidden_size": hidden_size,
                                    "batch_size": batch_size,
                                    "seq_len": seq_len,
                                    "resample_dt": resample_dt,
                                    "time_last_epoch": time_last_epoch,
                                    "computation_flops": computation_flops,
                                    "num_parameters": num_parameters,
                                    "model_type": model_type,
                                    "min_val_loss": min_val_loss,
                                    "val_baseline": val_baseline,
                                    "validation_dataset": row["dataset"],
                                    "validation_loss": row["validation_loss"],
                                    "validation_baseline": row["validation_baseline"],
                                }
                            )

    # List to store result dictionaries
    results = []
    recursive_search(results_root_dir, results)
    return pd.DataFrame(results)

In [ ]:
# Usage example:
results_root_directory = "/om2/vast/yanglab/shared/qsimeon/worm-graph-experiments/version_1"
combined_results_df = get_results_df(results_root_directory)
combined_results_df  # .head()  # To display the first few rows of the resulting DataFrame

# Scaling Number of Parameters

In [ ]:
# %%cudf.pandas.profile


# We varied the hidden size as a "knob" to vary the number of parameters.
def parameters_scaling_plot(combined_results_df, legend_code, title=None):
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_title(title, fontsize=14)

    # Group the DataFrame by 'model_type' and plot
    for model_type, group in combined_results_df.groupby("model_type"):
        # Skip if unrecognized type of model
        if model_type not in legend_code["model_labels"]:
            continue
        # Plot settings from legend_code
        model_label = legend_code["model_labels"][model_type]
        marker = legend_code["model_marker_code"][model_label]
        color = legend_code["model_color_code"][model_label]

        # Scatter plot of validation loss vs. number of parameters
        ax.scatter(
            group["num_parameters"],
            group["min_val_loss"],
            marker=marker,
            s=10,
            color=color,
            label=model_label,
            alpha=0.5,  # Adjust alpha to your preference
        )

    baseline = np.round(combined_results_df["val_baseline"], 3).unique()
    # DEBUG: There should be one baseline but we may have two if we ran two sets of experiments (train_split_first = True or False)
    for bsl in baseline:
        ax.plot(
            [
                combined_results_df["num_parameters"].min(),
                combined_results_df["num_parameters"].max(),
            ],
            [bsl, bsl],
            label="Baseline",
            color="black",
            alpha=0.7,
            linestyle="--",
        )

    # Create legends
    ax.legend(loc="best")
    ax.set_xlabel("Num. trainable parameters", fontsize=12, fontweight="bold")
    ax.set_ylabel("Minimum Validation Loss (MSE)", fontsize=12, fontweight="bold")
    plt.show()

`results_directory = "/om2/vast/yanglab/shared/qsimeon/worm-graph-experiments/version_1/num_parameters/experimental"`

In [ ]:
# Retrieve the relevant results
results_directory = "/om2/vast/yanglab/shared/qsimeon/worm-graph-experiments/version_1/num_parameters/experimental"  # real worms AllExperimental
results_df = get_results_df(results_directory)
results_df  # .head()

# Example usage:
parameters_scaling_plot(results_df, leg_code, title="All Experimental")

`results_directory = "/om2/vast/yanglab/shared/qsimeon/worm-graph-experiments/version_1/num_parameters/synthetic"`

In [ ]:
# # Retrieve the relevant results
# results_directory = "/om2/vast/yanglab/shared/qsimeon/worm-graph-experiments/version_1/num_parameters/synthetic/seed_0"  # synthetic data Sines
# results_df = get_results_df(results_directory)
# results_df  # .head()

# # Example usage:
# parameters_scaling_plot(results_df, leg_code, title="Sines")

In [ ]:
# # Retrieve the relevant results
# results_directory = "/om2/vast/yanglab/shared/qsimeon/worm-graph-experiments/num_parameters/synthetic/seed_1"  # synthetic data Lorenz
# results_df = get_results_df(results_directory)
# results_df  # .head()

# # Example usage:
# parameters_scaling_plot(results_df, leg_code, title="Lorenz")

In [ ]:
# # Retrieve the relevant results
# results_directory = "/om2/vast/yanglab/shared/qsimeon/worm-graph-experiments/num_parameters/synthetic/seed_2"  # synthetic data White Noise
# results_df = get_results_df(results_directory)
# results_df  # .head()

# # Example usage:
# parameters_scaling_plot(results_df, leg_code, title="White Noise")

In [ ]:
# # Retrieve the relevant results
# results_directory = "/om2/vast/yanglab/shared/qsimeon/worm-graph-experiments/num_parameters/synthetic/seed_3"  # synthetic data Random Walk
# results_df = get_results_df(results_directory)
# results_df  # .head()

# # Example usage:
# parameters_scaling_plot(results_df, leg_code, title="Random Walk")

In [ ]:
# # Retrieve the relevant results
# results_directory = "/om2/vast/yanglab/shared/qsimeon/worm-graph-experiments/num_parameters/synthetic/seed_5"  # synthetic data Shakespeare
# results_df = get_results_df(results_directory)
# results_df  # .head()

# # Example usage:
# parameters_scaling_plot(results_df, leg_code, title="Shakespeare")

In [ ]:
# %%cudf.pandas.profile


def print_best_model_info(df):
    """
    Prints the hidden size and number of parameters for the entry with the lowest min_val_loss
    for each unique model type in the DataFrame.

    Parameters:
    - df: A pandas DataFrame containing the model results.
    """
    model_types = df["model_type"].unique()
    for model_type in model_types:
        # Filter for the current model type
        model_df = df[df["model_type"] == model_type]

        # Find the index of the minimum validation loss
        idx_min_val_loss = model_df["min_val_loss"].idxmin()

        # Get the row for the minimum validation loss
        min_val_loss_row = model_df.loc[idx_min_val_loss]

        # Extract the hidden_size and num_parameters
        hidden_size = min_val_loss_row["hidden_size"]
        num_parameters = min_val_loss_row["num_parameters"]
        min_val_loss = min_val_loss_row["min_val_loss"]

        print(
            f"The {model_type} model achieved its lowest min_val_loss of {min_val_loss:.6f} "
            f"with a hidden_size of {hidden_size} and num_parameters of {num_parameters}."
        )


# Example usage:
print_best_model_info(results_df)

# Data Scaling

# Mixed Dataset Scaling

In [ ]:
# %%cudf.pandas.profile


def mixed_scaling_plot(combined_results_df, legend_code):
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    ax.set_xscale("log")
    ax.set_yscale("log")

    for model_type, model_group in combined_results_df.groupby("model_type"):
        # Skip if unrecognized type of model
        if model_type not in legend_code["model_labels"]:
            continue

        # Obtain values for plotting
        nts = model_group["num_time_steps"]
        val_loss = model_group["min_val_loss"]
        baseline_loss = model_group["val_baseline"].iloc[0]  # Baseline is constant

        # Plot settings from legend_code
        model_label = legend_code["model_labels"][model_type]
        marker = legend_code["model_marker_code"][model_label]
        color = legend_code["model_color_code"][model_label]

        # Scatter plot
        ax.scatter(
            nts,
            val_loss,
            marker=marker,
            color=color,  # TODO: is this preventing the alpha from kicking in?
            s=7,
            label=None,
            alpha=0.3,
        )

        # Baseline plot
        ax.axhline(
            baseline_loss,
            label="Baseline"
            if model_type == list(legend_code["model_labels"].values())[0]
            else "_nolegend_",
            color="black",
            alpha=0.7,
            linestyle="--",
        )

        # Regression line
        slope, intercept, r_value, p_value, std_err = stats.linregress(
            np.log(nts), np.log(val_loss)
        )
        fit_label = f"y = {slope:.2f}x + {intercept:.1f}\n" f"$R^2={r_value**2:.2f}$"
        x_fit = np.linspace(nts.min(), nts.max(), 10000)
        y_fit = np.exp(intercept + slope * np.log(x_fit))

        ax.plot(x_fit, y_fit, color=color, label=fit_label)

    # Handles and labels for first legend
    handles, labels = ax.get_legend_handles_labels()

    # Create the second legend
    legend2 = ax.legend(
        handles=legend_code["marker_legend"],
        loc="center right",
        bbox_to_anchor=(0.995, 0.85),
        title="Model architecture",
        fontsize="x-small",
    )
    ax.get_legend().get_title().set_fontstyle("italic")
    ax.get_legend().get_title().set_fontsize("x-small")
    ax.add_artist(legend2)

    # Display both legends
    ax.legend(handles, labels, loc="center left", bbox_to_anchor=(0.01, 0.25), fontsize="xx-small")

    ax.set_xlabel("Num. train time steps", fontsize=12)
    ax.set_ylabel("Validation MSE Loss", fontsize=12)

    plt.show()

`results_directory = "/om2/vast/yanglab/shared/qsimeon/worm-graph-experiments/version_1/num_time_steps/experimental"`

In [ ]:
# Retrieve the relevant results
results_directory = (
    "/om2/vast/yanglab/shared/qsimeon/worm-graph-experiments/version_1/num_time_steps/experimental"
)
results_df = get_results_df(results_directory)
results_df  # .head()

# Example usage
mixed_scaling_plot(results_df, leg_code)

# Individual Dataset Scaling

In [ ]:
# %%cudf.pandas.profile


def individual_scaling_plot(combined_results_df, legend_code):
    fig, axs = plt.subplots(1, len(combined_results_df["model_type"].unique()), figsize=(15, 5))

    if len(combined_results_df["model_type"].unique()) > 1:
        axs = axs.flatten()
    else:
        axs = [axs]

    for idx, (model_type, model_group) in enumerate(combined_results_df.groupby("model_type")):
        # Skip if unrecognized type of model
        if model_type not in legend_code["model_labels"]:
            continue

        model_label = legend_code["model_labels"][model_type]

        ax = axs[idx]
        ax.set_xscale("log")
        ax.set_yscale("log")

        for dataset_name, dataset_group in model_group.groupby("validation_dataset"):
            color = legend_code["dataset_color_code"][dataset_name]

            # Scatter plot for individual dataset
            ax.scatter(
                dataset_group["num_time_steps"],
                dataset_group["validation_loss"],
                s=7,
                color=color,
                label=dataset_name,
                alpha=0.3,
            )

            # Baseline horizontal line
            baseline_loss = dataset_group["validation_baseline"].iloc[0]
            ax.axhline(
                y=baseline_loss,
                color=color,
                linestyle="--",
                linewidth=1,
                alpha=0.7,
                label=f"{dataset_name} baseline",
            )

            # Linear regression
            slope, intercept, r_value, p_value, std_err = stats.linregress(
                np.log(dataset_group["num_time_steps"]),
                np.log(dataset_group["validation_loss"]),
            )

            # Plot regression line
            x_reg = np.linspace(
                dataset_group["num_time_steps"].min(),
                dataset_group["num_time_steps"].max(),
                100,
            )
            y_reg = np.exp(intercept + slope * np.log(x_reg))
            ax.plot(x_reg, y_reg, color=color)

        ax.set_title(model_label, fontweight="bold")
        ax.set_xlabel("Num. train time steps")
        if idx == 0:
            ax.set_ylabel("Validation Loss")
        else:
            ax.set_yticklabels([])

    # Adjust the layout and display the plot
    plt.tight_layout()
    plt.show()

`results_directory = "/om2/vast/yanglab/shared/qsimeon/worm-graph-experiments/version_1/num_time_steps/experimental"`

In [ ]:
# Retrieve the relevant results
results_directory = (
    "/om2/vast/yanglab/shared/qsimeon/worm-graph-experiments/version_1/num_time_steps/experimental"
)
results_df = get_results_df(results_directory)
results_df.head()

# Example usage:
individual_scaling_plot(results_df, leg_code)

# TODO: Cross Dataset Generalization
A heatmap showing the validation loss on all of the experimental datasets after a model has been trained on just one of the experimental datasets.

In [ ]:
# # 5. Cross-dataset Generalization plot
# def cross_dataset(experiment_log_folders, model_names, legend_code):
#     dataset_labels = legend_code["dataset_labels"]

#     analysis_df = pd.DataFrame(
#         columns=[
#             "experiment_ID",
#             "model_type",
#             "train_dataset",
#             "val_dataset",
#             "val_loss",
#             "val_baseline",
#         ]
#     )

#     for exp_log_dir, model in zip(experiment_log_folders, model_names):
#         for experiment_ID in sorted(
#             os.listdir(exp_log_dir), key=lambda x: x.strip("exp_")
#         ):
#             # Skip if not starts with exp
#             if not experiment_ID.startswith("exp") or experiment_ID.startswith("exp_"):
#                 continue

#             val_url = os.path.join(
#                 exp_log_dir,
#                 experiment_ID,
#                 "analysis",
#                 "validation_loss_per_dataset.csv",
#             )
#             train_ds_url = os.path.join(
#                 exp_log_dir, experiment_ID, "dataset", "train_dataset_info.csv"
#             )

#             val_df = pd.read_csv(val_url)
#             val_df["experiment_ID"] = experiment_ID
#             val_df["model_type"] = model

#             train_dataset_info = pd.read_csv(train_ds_url)
#             val_df["train_dataset"] = train_dataset_info["dataset"].unique()[0]

#             # Change 'dataset' column name to 'val_dataset'
#             val_df = val_df.rename(columns={"dataset": "val_dataset"})

#             # Swap uzel and kaplan
#             val_df = val_df.iloc[[0, 1, 2, 4, 3, 5, 6], :]
#             val_df = val_df.reset_index(drop=True)

#             analysis_df = pd.concat([analysis_df, val_df], axis=0)

#     train_ds_names = [
#         "Leifer2023",
#         "Flavell2023",
#         "Uzel2022",
#         "Yemini2021",
#         "Kaplan2020",
#         "Skora2018",
#         "Nichols2017",
#         "Kato2015",
#     ]
#     val_ds_names = analysis_df["val_dataset"].unique()
#     models = analysis_df["model_type"].unique()

#     # Figure size
#     fig, ax = plt.subplots(1, len(models), figsize=(12, 4), sharex="col", sharey="row")

#     # Initialize vmin and vmax for the color scale
#     vmin = analysis_df["val_loss"].min()
#     vmax = analysis_df["val_loss"].max()

#     for i, model_name in enumerate(models):
#         # Filter data for the specific model
#         df_model_subset = analysis_df[analysis_df["model_type"] == model_name]

#         # Create an empty matrix for the heatmap data
#         heatmap_data = pd.DataFrame(columns=train_ds_names, index=val_ds_names)

#         for train_ds in train_ds_names:
#             for val_ds in val_ds_names:
#                 value = df_model_subset[
#                     (df_model_subset["train_dataset"] == train_ds)
#                     & (df_model_subset["val_dataset"] == val_ds)
#                 ]["val_loss"].values
#                 if value:
#                     heatmap_data.at[val_ds, train_ds] = value[0]

#         # Plot the heatmap
#         sns.heatmap(
#             heatmap_data.astype(float),
#             cmap="magma_r",
#             ax=ax[i],
#             annot=True,
#             square=True,
#             cbar=False,
#             vmin=vmin,
#             vmax=vmax,
#         )
#         ax[i].set_title("{}".format(model_name), fontsize=16)
#         # Set xlabel
#         ax[i].set_xlabel("Train dataset", fontsize=14, fontweight="bold")
#         ax[i].set_xticklabels(dataset_labels, rotation=90, fontsize=10)
#         # Set ylabel
#         ax[0].set_ylabel("Validation dataset", fontsize=14, fontweight="bold")
#         ax[i].set_yticklabels(dataset_labels, rotation=0, fontsize=10)

#     # Add a single colorbar at the rightmost part
#     cbar_ax = fig.add_axes(
#         [0.92, 0.125, 0.02, 0.755]
#     )  # [left, bottom, width, height] of the colorbar axes in figure coordinates.
#     fig.colorbar(
#         ax[-1].collections[0], cax=cbar_ax
#     )  # ax[-1].collections[0] grabs the colormap of the last subplot

#     # Add title to cmap
#     cbar_ax.set_ylabel(
#         "Validation loss (MSE)",
#         fontsize=12,
#         fontweight="bold",
#         rotation=90,
#         labelpad=-57,
#     )

#     plt.tight_layout(pad=0.1)
#     plt.subplots_adjust(
#         right=0.9
#     )  # adjust the rightmost part to make room for the colorbar
#     plt.show()

In [ ]:
# experiment_log_folders = [
#     os.path.join("results", "CDS_LINEAR"),  # Linear trial 1
#     os.path.join("results", "CDS_LSTM"),  # LSTM trial 1
#     os.path.join("results", "CDS_TRANSFORMER"),  # Transformer trial 1
# ]

# model_names = ["Feedforward", "LSTM", "Transformer"]

# cross_dataset(
#     experiment_log_folders=experiment_log_folders,
#     model_names=model_names,
#     legend_code=leg_code,
# )